In [10]:
from tensorflow import keras
from keras import layers
import os
import numpy as np
import matplotlib

In [34]:
original_dim = (256,256,3)
intermediate_dim = 64
latent_dim = 2

In [66]:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

inputs = keras.Input(shape=original_dim)

# Create/Apply Conv Layers
bn1 = layers.BatchNormalization()(inputs)
c1 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn1)
mp1 = layers.MaxPooling2D((4, 4), padding='same')(c1)
dpo1 = layers.Dropout(0.3)(mp1)

bn2 = layers.BatchNormalization()(dpo1)
c2 = layers.Conv2D(16, (4,4), activation='relu', padding='same')(bn2)
mp2 = layers.MaxPooling2D((4, 4), padding='same')(c2)
dpo2 = layers.Dropout(0.3)(mp2)

bn3 = layers.BatchNormalization()(dpo2)
c3 = layers.Conv2D(8, (8,8), activation='relu', padding='same')(bn3)
mp3 = layers.MaxPooling2D((2, 2), padding='same')(c3)
dpo3 = layers.Dropout(0.3)(mp3)
print(dpo3.shape)

# Flatten the layer
flat = layers.Flatten()(dpo3)
print(flat.shape)
h = layers.Dense(intermediate_dim, activation='relu')(flat)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)
z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
x = layers.Dense(512, activation='relu')(x)

reshape = layers.Reshape((8,8,8))(x) # undo flatten
# Deconvolution
bn4 = layers.BatchNormalization()(reshape)
c4 = layers.Conv2D(8, (8,8), activation='relu', padding='same')(bn4)
us4 = layers.UpSampling2D((4, 4))(c4)
dpo4 = layers.Dropout(0.3)(us4)

bn5 = layers.BatchNormalization()(dpo4)
c5 = layers.Conv2D(16, (4,4), activation='relu', padding='same')(bn5)
us5 = layers.UpSampling2D((2, 2))(c5)
dpo5 = layers.Dropout(0.3)(us5)

bn6 = layers.BatchNormalization()(dpo5)
c6 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn6)
us6 = layers.UpSampling2D((4, 4))(c6)
dpo6 = layers.Dropout(0.3)(us6)

# bn7 = layers.BatchNormalization()(dpo6)
# c7 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn7)
# us7 = layers.UpSampling2D((4, 4))(c7)
# dpo7 = layers.Dropout(0.3)(us7)

# bn8 = layers.BatchNormalization()(dpo7)
# c8 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn8)
# us8 = layers.UpSampling2D((4, 4))(c8)
# dpo8 = layers.Dropout(0.3)(us8)

outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(dpo6)


# outputs = layers.Dense(original_dim[1], activation='sigmoid')()






decoder = keras.Model(latent_inputs, outputs, name='decoder')





# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


(None, 8, 8, 8)
(None, 512)


In [67]:
# Create the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim[1]
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
print(vae.summary())

In [28]:
from PIL import Image
subreddits = os.listdir("Data")
for subreddit in subreddits:
    subreddit_folder = os.path.join("Data", subreddit)
    image_paths = [os.path.join(subreddit_folder, filename) for filename in os.listdir(subreddit_folder) if ".csv" not in filename]
    for image_path in image_paths:
        image = Image.open(image_path)
        w = image.width
        h = image.height
        crop = min(w, h)
        crop_dim = (w//2 - crop//2, h//2 - crop//2, w//2 + crop//2, h//2 + crop//2)
        image = image.crop(crop_dim)
        image.thumbnail(original_dim)
        # image.show()
        image_array = np.asarray(image)
        print(image_array.shape)
        

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data'